#  주택청약 FAQ 시스템 챗봇 구현 

- 문서 전처리 + RAG + Gradio ChatInterface

## 1. 환경 설정

In [1]:
# 환경 변수
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# 기본 라이브러리
import os
from glob import glob

from pprint import pprint
import json

In [3]:
# LLM 설정
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.1,
    top_p=0.8, 
)

## 2. 문서 전처리

* 데이터 정제 : 원본 문서에서 HTML 태그, 특수문자, 중복 문장 등을 제거하고 텍스트를 표준화하여 검색 품질을 높임
* 문서 청킹(Chunking) : 문서를 문맥이 유지되도록 문장 또는 단락 단위로 분할하여 검색 정확도와 응답 품질을 향상시킴
* 임베딩(Embedding) : 텍스트를 의미 기반 고차원 벡터로 변환하여 유사도 기반 검색이 가능하도록 함
* 벡터 데이터베이스 색인화 : 임베딩된 벡터를 벡터 데이터베이스에 색인화하여 대규모 문서에서도 빠른 검색을 가능하게 함

### 1) 문서 로드

- 국토교통부 주택청약 FAQ에서 일부 내용(청약자격, 청약통장)을 발췌하여 재가공한 문서로, 50개의 문답이 포함된 텍스트 파일임

In [4]:
# 여기에 코드를 작성하세요.
from langchain_community.document_loaders import TextLoader

loader = TextLoader(
    file_path="data/housing_faq.txt",
    encoding="utf-8"
)
docs = loader.load()

In [5]:
# 문서 확인
print(docs[0].page_content[:500])

Q1 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
A 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 
참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Q2 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
A 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다.
* 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원
다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 


In [6]:
# 문서 메타데이터 확인
docs[0].metadata

{'source': 'data/housing_faq.txt'}

### 2) 문서 전처리

(1) 정규표현식을 활용하여 문서에서 질문-답변 쌍을 구조적으로 분리함

In [7]:
import re

def extract_qa_pairs(text):
    qa_pairs = []
    
    # 텍스트를 라인별로 분리하고 각 라인의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    current_question = None
    current_answer = []
    current_number = None
    in_answer = False
    
    for i, line in enumerate(lines):
        if not line:  # 빈 라인 처리
            if in_answer and current_answer and i + 1 < len(lines) and lines[i + 1].startswith('Q'):
                # 다음 질문이 시작되기 전 빈 줄이면 현재 QA 쌍 저장
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
                in_answer = False
                current_answer = []
            continue
            
        # 새로운 질문 확인 (Q 다음에 숫자가 오는 패턴)
        q_match = re.match(r'Q(\d+)\s+(.*)', line)
        if q_match:
            # 이전 QA 쌍이 있으면 저장
            if current_question is not None and current_answer:
                qa_pairs.append({
                    'number': current_number,
                    'question': current_question,
                    'answer': ' '.join(current_answer).strip()
                })
            
            # 새로운 질문 시작
            current_number = int(q_match.group(1))
            current_question = q_match.group(2).strip().rstrip('?') + '?'  # 질문 마크 정규화
            current_answer = []
            in_answer = False
            
        # 답변 시작 확인
        elif line.startswith('A ') or (current_question and not current_answer and line):
            in_answer = True
            current_answer.append(line.lstrip('A '))
            
        # 기존 답변에 내용 추가
        elif current_question is not None and (in_answer or not line.startswith('Q')):
            if in_answer or (current_answer and not line.startswith('Q')):
                current_answer.append(line)
    
    # 마지막 QA 쌍 처리
    if current_question is not None and current_answer:
        qa_pairs.append({
            'number': current_number,
            'question': current_question,
            'answer': ' '.join(current_answer).strip()
        })
    
    # 번호 순서대로 정렬
    qa_pairs.sort(key=lambda x: x['number'])
    
    return qa_pairs

In [8]:
# QA 쌍 추출
qa_pairs = extract_qa_pairs(docs[0].page_content) 

print(f"추출된 QA 쌍 개수: {len(qa_pairs)}")
print(f"추출된 첫번째 QA: \n{qa_pairs[0]}")

추출된 QA 쌍 개수: 50
추출된 첫번째 QA: 
{'number': 1, 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?', 'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.'}


(2) LLM을 활용하여 텍스트에서 키워드와 핵심 개념을 추출하고, 이를 메타데이터나 본문에 추가하여 검색 성능을 향상시킴

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 출력 형식 정의
class KeywordOutput(BaseModel):
    keyword: str = Field(description="질문 텍스트에서 추출한 가장 중요한 키워드")
    summary: str = Field(description="질문과 응답 텍스트의 핵심 내용 요약")

# 프롬프트 템플릿 정의
template = """주어진 질문 텍스트에서 핵심 단어를 추출하고, 응답 텍스트의 핵심 내용을 요약합니다.
질문 및 응답 텍스트의 맥락을 고려하여 핵심 용어나 전문 용어, 주요 아이디어나 원리 등을 추출합니다.

질문 텍스트:
{question}

응답 텍스트:
{answer}

JSON 형식으로 다음 정보를 반환하시오:
- keyword: 단어 1개 만
- summary: 1-2 문장 만
"""

# LCEL 체인 구성 (Sturctured Output 사용)
prompt = ChatPromptTemplate.from_template(template)
llm_with_structure = llm.with_structured_output(KeywordOutput)
keyowrd_extractor = prompt | llm_with_structure

# 텍스트 추출 테스트     
result = keyowrd_extractor.invoke({
    "question": qa_pairs[36]['question'],
    "answer": qa_pairs[36]['answer']
})
print("키워드:", result.keyword)
print("요약:", result.summary)

키워드: 입주자저축
요약: 제2순위 청약신청 시 입주자저축 통장이 필요하며, 분양주택 또는 분양전환공공임대주택 입주자로 선정된 경우 해당 통장은 재사용할 수 없습니다.


(3) 요약문을 시맨틱 검색에 활용하기 위해, 요약을 `page_content`에 담고 기타 정보를 `metadata`로 구성한 문서 객체를 생성함.

In [10]:
from langchain_core.documents import Document

def format_qa_pairs(qa_pairs):
    """
    추출된 QA 쌍을 포맷팅하여 문서 객체로 변환
    """
    processed_docs = []
    for pair in qa_pairs:
        # 키워드와 요약 추출
        result = keyowrd_extractor.invoke({
            "question": pair['question'],
            "answer": pair['answer']
        })

        # 문서 객체 생성
        doc = Document(
            page_content=result.summary,
            metadata={
                'question_id': int(pair['number']),
                'question': pair['question'],
                'answer': pair['answer'],
                'keyword': result.keyword,
            }
        )
        processed_docs.append(doc)

    return processed_docs


# QA 쌍 포맷팅
formatted_docs = format_qa_pairs(qa_pairs)
print(f"포맷팅된 문서 개수: {len(formatted_docs)}")

# 문서 확인
print(formatted_docs[5].page_content)
print("-" * 200)
# 문서 메타데이터 확인
pprint(formatted_docs[5].metadata)

포맷팅된 문서 개수: 50
대규모택지개발지구에서 주택 공급 시 특별공급 물량도 공급 비율에 따라 배정되며, 거주지별 우선공급 비율이 적용된다. 다자녀 특별공급은 별도의 운용지침에 따라 시·도 단위로 우선공급이 이루어진다.
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
{'answer': '공급규칙 제34조가 적용되는 지역에 주택을 공급하는 경우 특별공급 물량 또한 그 공급 비율에 따라 배정하여야 합니다. '
           '(예) 서울특별시·인천광역시 대규모 택지개발지구에서 공급되는 주택: 특별시·광역시 거주자(거주기간 충족 필요)에게 '
           '공급물량의 50% 우선공급 → 이후 잔여물량을 전체 수도권 거주자를 대상으로 공급 (예) 경기도 과천시 대규모 '
           '택지개발지구에서 공급되는 주택: 과천시 거주자(거주기간 충족 필요)에게 공급물량의 30% 우선공급 → 이후 과천시 공급 '
           '잔여물량 + 20% 경기도 거주자(거주기간 충족 필요)에게 공급 → 이후 잔여물량을 전체 수도권 거주자를 대상으로 공급 '
           '다만, 다자녀 특별공급의 경우 「다자녀가구 및 노부모부양 주택 특별공급 운용지침」 제5조 단서에 따라 수도권에서 '
           '입주자를 모집하는 때에는 해당 주택건설지역 시·군·구가 속한 시·도에 50퍼센트를 우선공급하고 나머지 주택(우선공급에서 '
           '미분양된 주택을 포함한다)은 수도권 거주자(우선공급에서 입주자로 선정되지 아니한 자를 포함한다)에게 공급할수 있습니다. '
           '(예) 경기도 과천시: 경기도 거주자에게 공급물량의

In [11]:
# 문서 저장
output_file = "data/housing_faq_formatted.json"
with open(output_file, 'w', encoding='utf-8-sig') as f:
    json.dump(
        [doc.model_dump() for doc in formatted_docs], 
        f, 
        indent=2, 
        ensure_ascii=False
    )  # 한글이 유니코드로 변환되지 않도록 설정

print(f"포맷팅된 문서를 {output_file}에 저장했습니다.")

포맷팅된 문서를 data/housing_faq_formatted.json에 저장했습니다.


In [6]:
# 문서 로드
from langchain_core.documents import Document

output_file = "data/housing_faq_formatted.json"
with open(output_file, 'r', encoding='utf-8-sig') as f:
    formatted_docs = [Document(**doc) for doc in json.load(f)]
    
# 문서 확인
print(formatted_docs[0].page_content)
pprint(formatted_docs[0].metadata)

[1]
질문: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
답변: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

{'answer': '해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 '
           '말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 '
           '주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 '
           '전역이 해당 주택건설지역에 해당됩니다.',
 'keyword': '주택건설지역',
 'question': '경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?',
 'question_id': 1,
 'summary': '해당 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 '
            '공급되는 주택의 경우 과천시가 해당 지역에 해당한다.'}


### 3\) 문서 벡터 저장 및 색인화

In [ ]:
# 컬렉션이 존재하는 경우 Drop
from pymilvus import connections, utility

connections.connect(alias="default", uri="./milvus.db")

if utility.has_collection("housing_faq_db"):
    utility.drop_collection("housing_faq_db")
    print("컬렉션 삭제 완료")
else:
    print("컬렉션이 존재하지 않습니다.")

컬렉션 삭제 완료


In [6]:
from langchain_milvus import Milvus
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name='housing_faq_db',
    connection_args={"uri": "./milvus.db"},
    # 인덱싱 구조 정의
    index_params={
        "index_type": "IVF_FLAT", # 벡터를 여러 클러스터로 나눈 뒤, 각 클러스터에서 FLAT 검색
        "metric_type": "COSINE",
        "params": {"nlist": 128}
    },
)

/Users/sunhwaryu/Documents/LangChain/_jupyter/.venv/lib/python3.12/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
2025-06-02 00:32:24,964 [DEBUG][_create_connection]: Created new connection using: 8d3d908945ea4f96a296c262fc21d97f (async_milvus_client.py:599)


In [ ]:
# 문서 Insert
ids = vector_store.add_documents(documents=formatted_docs)
len(ids)

/Users/sunhwaryu/Documents/LangChain/_jupyter/.venv/lib/python3.12/site-packages/langchain_milvus/vectorstores/milvus.py:1141: UserWarning: No ids provided and auto_id is False. Setting auto_id to True automatically.
  warnings.warn(


50

## 2. 문서 검색

In [26]:
# 메타데이터 기반 필터링
retriever = vector_store.as_retriever(
    search_kwargs={"expr": 'keyword == "주택건설지역"'},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print("=" * 50)

주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 주택건설지역에 해당한다.
--------------------------------------------------
주택건설지역
1


In [82]:
# MMR 검색기 정의 및 테스트
mmr_retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.5},
)

query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = mmr_retriever.invoke(query)
for result in results:
    print(result.page_content)
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 주택건설지역에 해당한다.
--------------------------------------------------
주택건설지역
1
경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.
무주택세대구성원이란 청약신청자와 세대원 모두가 주택을 소유하지 않은 세대의 구성원을 의미한다.
--------------------------------------------------
무주택세대구성원
47
무주택세대구성원이란?
무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.
서울시에서 전용면적 102㎡ 이하 주택에 청약하려면 예치금액이 600만원이어야 하므로, 기존 400만원에서 200만원을 추가 예치해야 한다.
--------------------------------------------------
청약예금
39
인천광역시 거주자로 청약예금 400만원(전용면적 102㎡ 이하)에 가입한 자가 입주자 모집공고일 전 서울시로 이주한 경우 102㎡ 이하의 주택에 청약하려면?
서울의 경우 전용면적 102㎡ 이하 주택에 청약할 수 있는 예치금액은 600만원이기 때문에 청약접수 당일까지 부족금액인 200만원을 추가로 예치하여야만 102㎡ 이하의 주택에 청약이 가능


In [ ]:
# Rerank 검색 테스트
query = "수원시의 주택건설지역은 어디에 해당하나요?"

results = vector_store.similarity_search_with_score(
    query, 
    k=3, 
    ranker_type="weighted", 
    ranker_params={"weights": [0.7, 0.3]},
)
for result, score in results:
    print(result.page_content)
    print([score])
    print("-" * 50)
    print(result.metadata['keyword'])
    print(result.metadata['question_id'])
    print(result.metadata['question'])
    print(result.metadata['answer'])
    print("=" * 50)

주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 주택건설지역에 해당한다.
[0.4981459081172943]
--------------------------------------------------
주택건설지역
1
경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.
해당 주택건설지역에 거주하지 않아도 청약가능지역 내 주택에 청약신청이 가능하나, 같은 순위에서는 해당 지역 거주자가 우선 공급받으며, 일부 대규모 택지개발지구에서는 거주자와 동등한 자격으로 공급받을 수 있다.
[0.39675116539001465]
--------------------------------------------------
청약신청
2
해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.
무주택세대구성원이란 청약신청자와 세대원 모두가 주택을 소유하지 않은 세대의 구성원을 의미한다.
[0.3937966823577881]
----------

## 3. RAG Chain

### 1\) Retriever 생성

In [4]:
def get_relevant_documents(query):
    """Context 반환"""
    results = vector_store.similarity_search(
        query, 
        k=3, 
        ranker_type="weighted", 
        ranker_params={"weights": [0.6, 0.4]}
    )
    context = []
    for result in results:
        s = []
        s.append(f'Document {result.metadata['question_id']}:')
        s.append(f'- keyword: {result.metadata['keyword']}')
        s.append(f'- summary: {result.page_content}')
        s.append(f'- question: {result.metadata['question']}')
        s.append(f'- answer: {result.metadata['answer']}')
        context.append('\n'.join(s))

    return '\n\n'.join(context)

In [7]:
query = "수원시의 주택건설지역은 어디에 해당하나요?"
context = get_relevant_documents(query)
print(context)

Document 1:
- keyword: 주택건설지역
- summary: 주택건설지역은 특별시, 광역시, 특별자치시, 특별자치도 또는 시·군의 행정구역을 의미하며, 경기도 과천시에서 공급되는 주택의 경우 과천시가 해당 주택건설지역에 해당한다.
- question: 경기도 과천시에서 공급되는 주택의 해당 주택건설지역의 범위는?
- answer: 해당 주택건설지역이란 특별시ㆍ광역시ㆍ특별자치시ㆍ특별자치도(관할 구역 안에 지방자치단체인 시ㆍ군이 없는 특별자치도를 말한다) 또는 시ㆍ군의 행정구역을 말합니다. 따라서, 경기도 과천시에서 공급하는 주택의 경우 과천시가 해당 주택건설지역에 해당됩니다. 참고로, 서울특별시에서 공급되는 주택의 경우 서울특별시 전역, 인천광역시의 경우 인천광역시 전역이 해당 주택건설지역에 해당됩니다.

Document 2:
- keyword: 청약신청
- summary: 해당 주택건설지역에 거주하지 않아도 청약가능지역 내 주택에 청약신청이 가능하나, 같은 순위에서는 해당 지역 거주자가 우선 공급받으며, 일부 대규모 택지개발지구에서는 거주자와 동등한 자격으로 공급받을 수 있다.
- question: 해당 주택건설지역에 거주하고 있지 않다면 청약신청이 불가능한지?
- answer: 해당 주택건설지역에 거주하고 있지 않더라도 청약가능지역에서 공급되는 주택에 청약신청이 가능하나, 같은 순위에서는 해당 주택건설지역의 거주자가 우선하여 주택을 공급받게 됩니다. * 서울·인천·경기도 / 대전·세종·충남 / 충북 / 광주·전남 / 전북 / 대구·경북 / 부산·울산·경남 / 강원 다만, 수도권 대규모 택지개발지구 등에서 주택이 공급되는 경우 일정 비율의 주택에 대해서는 해당 주택건설지역 거주자와 동등한 자격으로 주택을 공급받을 기회를 가지게 됩니다.

Document 47:
- keyword: 무주택세대구성원
- summary: 무주택세대구성원이란 청약신청자와 세대원 모두가 주택을 소유하지 않은 세대의 구성원을 의미한다.
- question: 무주택세대구성원이란

### 2) 참조 문서 기반 답변 생성

In [8]:
from langchain_core.output_parsers import StrOutputParser
from typing import Dict

def generate_answer(input_data: Dict) -> Dict:
    """컨텍스트와 질문을 입력으로 받아 답변을 생성

    Args:
        input_data (Dict): 컨텍스트와 질문이 포함된 딕셔너리

    Returns:
        Dict: 생성된 답변과 소스 문서 정보가 포함된 딕셔너리
    """

    template = "[Source] 내용을 기반으로 [Question]에 대한 답변을 한국어로 하세요.\n\n[Source]\n{context}\n\n[Question]{question}"
    answer_chain = ChatPromptTemplate.from_template(template) | llm | StrOutputParser()

    return {
        "question": input_data["question"], # 질문
        "answer": answer_chain.invoke(input_data),  # 생성된 답변 
        "source": input_data['context']  # 소스 문서 정보 
    }

In [9]:
from langchain_core.runnables import RunnableLambda, RunnableMap, RunnablePassthrough

# Chain 구성
rag_chain = (
    RunnableMap(
        question=RunnablePassthrough(),
        context=RunnableLambda(get_relevant_documents) # Source 가져오기
    ) | 
    RunnableLambda(generate_answer) # 답변 생성
)

In [41]:
# Chain 실행
query = "무주택 세대에 대해 설명하세요."
result = rag_chain.invoke(query)

# 결과 출력
print("답변:", result["answer"])
print("\n참조 문서:\n", result["source"])

답변: 무주택 세대란 청약신청자와 그 세대원 모두가 주택을 소유하지 않은 세대를 의미합니다. 즉, 세대주를 포함한 세대 구성원 전원이 주택을 보유하고 있지 않은 경우를 말합니다. 또한, 세대분리된 직계비속의 배우자(예: 사위)는 친정부모의 세대원 범위에 포함되지 않으므로, 이 경우 친정부모가 무주택자라면 무주택세대구성원 자격이 인정됩니다.

참조 문서:
 Document 47:
- keyword: 무주택세대구성원
- summary: 무주택세대구성원이란 청약신청자와 세대원 모두가 주택을 소유하지 않은 세대의 구성원을 의미한다.
- question: 무주택세대구성원이란?
- answer: 무주택세대구성원이란 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다.

Document 50:
- keyword: 무주택세대구성원
- summary: 친정부모의 세대원 범위에는 세대분리된 직계비속의 배우자가 포함되지 않으므로, 무주택자인 친정부모는 무주택세대구성원 자격이 인정됩니다.
- question: 무주택자인 아내가 유주택자인 남편과 주민등록표상 분리되어 친정부모의 세대별 주민등록표에 등재되어 있는 경우, 무주택자인 친정부모는 무주택세대구성원 자격이 인정되는지?
- answer: 친정부모의 세대원 범위에 세대분리된 직계비속의 배우자(사위)는 포함되지 않으므로 사위가 주택을 소유하고 있다 하더라도 무주택세대구성원으로 인정됩니다.

Document 31:
- keyword: 소득공제
- summary: 청년주택드림청약통장은 기존 주택청약종합저축과 동일하게 연소득 7천만원 이하 무주택세대주가 연간 납입액 300만원 한도로 40%까지 소득공제를 받을 수 있습니다.
- question: 청년주택드림청약통장의 소득공제 혜택은 기존 주택청약종합저축과 동일한가요?
- answer: 현재 주택청약종합저축에서 제공하는 소득공제 조건(조세특례제한법 제87조)을 그대로 적용받게 되며, 연소득 7천만원 이하 무주택세대주로 무주택확인서를 제출하는 경우 

### 3) 검색 문서 관련성 평가

In [42]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 [소스]가 [질문]에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

1. [소스]가 [질문]에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
2. [소스]의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
3. [소스]의 정보가 [질문]에 대한 답변을 제공할 수 있는가?

'Yes' 또는 'No'로만 답변하세요."""),
    ("human", """[소스]
{source}

[질문]
{question}""")
])

relevance_chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용

relevance = relevance_chain.invoke({
    "source": result['source'],
    "question": "무주택 세대에 대해 설명하세요."
}).lower()

print(f"평가 결과: {relevance}") # 7초 소요

평가 결과: step 1: identify the question and what information is needed.
- question: "무주택 세대에 대해 설명하세요." (explain about 무주택 세대)
- needed information: definition or explanation of "무주택 세대" (households without housing ownership).

step 2: review the provided sources for relevant information.
- document 47 defines "무주택세대구성원" as members of a household where neither the applicant nor any household member owns a house.
- document 50 discusses the scope of household members in relation to 무주택세대구성원, specifically about separated family members and their recognition.
- document 31 relates to income deduction benefits for 무주택세대주 (head of household without housing ownership), but does not define the term.

step 3: evaluate if the sources contain the necessary information.
- document 47 directly defines "무주택세대구성원," which is closely related to "무주택 세대" (a household without housing ownership).
- document 50 provides additional details about household member scope but is more specific.
- document 31 is ab

In [ ]:
llm_gpt4o = ChatOpenAI(
    model='gpt-4.1',
    temperature=0.1,
    top_p=0.9, 
)

relevance_chain = prompt | llm_gpt4o | StrOutputParser()    # gpt-4.1 모델 사용

relevance = relevance_chain.invoke({
    "source": result['source'],
    "question": query
}).lower()

print(f"평가 결과: {relevance}") # 3.8초 소요, 엄격함

평가 결과: no


## 4. Gradio 챗봇 인터페이스

In [15]:
import gradio as gr
from langchain_core.language_models import BaseChatModel
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import List, Optional
from dataclasses import dataclass

In [17]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field

# 메모리 기반 히스토리 구현
class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        """Add a list of messages to the store"""
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

# 세션 저장소
store = {}

# 세션 ID로 히스토리 가져오기
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

# 히스토리 지우기
def remove_session_history(session_id: str) -> None:
    if session_id in store:
        store[session_id].clear()

In [56]:
def get_streaming_response(message: str, history) -> str:
    context = get_relevant_documents(message)

    prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 [소스]가 [질문]에 답변하는데 필요한 정보를 포함하고 있는지 논리적으로 평가하세요.
    단계적으로 진행하며, 평가결과에 대한 검증을 수행하세요.

    다음 기준 중 하나 이상을 충족할 경우 'Yes'로 답변하고, 모두 충족하지 못하면 'No'로 답변하세요:

    1. [소스]가 [질문]에 답변하는데 필요한 정보를 직접적으로 포함하고 있는가?
    2. [소스]의 정보로부터 답변에 필요한 내용을 논리적으로 추론할 수 있는가?
    3. [소스]의 정보가 [질문]에 대한 답변을 제공할 수 있는가?

    'Yes' 또는 'No'로만 답변하세요."""),
        ("human", """[소스]
    {source}

    [질문]
    {question}""")
    ])
    relevance_chain = prompt | llm | StrOutputParser()    # gpt-4.1-mini 모델 사용
    relevance = relevance_chain.invoke({
        "source": context,
        "question": message
    }).lower()

    if 'yes' in relevance:
        prompt = ChatPromptTemplate.from_messages([
            ("system", "다음 [Source] 내용을 참고해 [Question]에 한국어로 친절히 답하세요."),
            MessagesPlaceholder(variable_name="history"),
            ("human", "[Source]\n"+ context + "\n\n[Question]\n{question}")
            # ("human", "{question}")
        ])
        rag_chain_with_history = RunnableWithMessageHistory(
            prompt | llm | StrOutputParser(),
            get_session_history,
            input_messages_key="question",
            history_messages_key="history" # 히스토리 관리 추가 
        )
        response = rag_chain_with_history.invoke(
            {"question": message},
            config={"configurable": {"session_id": "user_1"}}
        )
        return response
    else: 
        return "죄송합니다. 관련 문서를 찾을 수 없어 답변하기 어렵습니다. 다른 질문을 해주시겠습니까?"

In [57]:
# 챗봇 인터페이스 생성
with gr.Blocks() as demo:
    
    gr.ChatInterface(
        fn=get_streaming_response,
        title="RAG 주택 청약 FAQ 시스템",
        description="""
        질문을 입력하면 관련 문서를 검색하여 답변을 생성합니다.
        모든 답변에는 참조한 문서의 출처가 표시됩니다.
        """,
        theme=gr.themes.Soft(
            primary_hue="blue",
            secondary_hue="gray",
        ),
        examples=[
            ["수원시의 주택건설지역은 어디에 해당하나요?"],
            ["무주택 세대에 대해서 설명해주세요."],
            ["2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?"],
        ],
        type="messages"
    )
    clear_button = gr.Button(value="이력 삭제")
    clear_button.click(fn=lambda _: remove_session_history("user_1"))

# 데모 실행
demo.launch()

/Users/sunhwaryu/Documents/LangChain/_jupyter/.venv/lib/python3.12/site-packages/gradio/utils.py:1028: UserWarning: Expected 1 arguments for function <function <lambda> at 0x17d8f8fe0>, received 0.
  warnings.warn(
/Users/sunhwaryu/Documents/LangChain/_jupyter/.venv/lib/python3.12/site-packages/gradio/utils.py:1032: UserWarning: Expected at least 1 arguments for function <function <lambda> at 0x17d8f8fe0>, received 0.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [60]:
# Gradio 인터페이스 종료
demo.close()

Closing server running on port: 7860


In [61]:
store

{'user_1': InMemoryHistory(messages=[HumanMessage(content='2순위로 당첨된 사람이 청약통장을 다시 사용할 수 있나요?', additional_kwargs={}, response_metadata={}), AIMessage(content='제2순위로 당첨된 경우에는 입주자저축 통장을 사용하게 되는데, 분양주택 또는 분양전환공공임대주택 입주자로 선정된 이후에는 해당 통장을 재사용할 수 없습니다. 따라서 2순위로 당첨된 사람은 이미 사용한 청약통장을 다시 사용할 수 없으니 참고하시기 바랍니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='무주택 세대에 대해서 설명해주세요.', additional_kwargs={}, response_metadata={}), AIMessage(content='무주택 세대란 청약신청자와 그 세대원 모두가 주택을 소유하지 않은 세대를 의미합니다. 즉, 무주택 세대구성원은 청약신청자 및 세대원 전원이 주택을 소유하고 있지 않은 세대의 구성원(세대주 포함)을 말합니다. 예를 들어, 친정부모가 무주택자이고, 세대분리된 직계비속의 배우자가 별도로 세대에 포함되지 않는 경우에는 친정부모는 무주택세대구성원 자격이 인정됩니다.', additional_kwargs={}, response_metadata={})])}